In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import torch


In [30]:
# llm = Ollama(model="mistral",  callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),num_gpu=1, base_url="http://localhost:11434")

from langchain import HuggingFaceHub
import torch

# Load the model and tokenizer
repo_id0 = "meta-llama/Meta-Llama-3-8B-Instruct"
repo_id1 = "mistralai/Mistral-7B-Instruct-v0.2"
repo_id2 = "google/gemma-7b"
model_kwargs = {"torch_dtype": str(torch.float16)}  # Convert torch.float16 to a string
llm = HuggingFaceHub(repo_id=repo_id2, model_kwargs=model_kwargs, huggingfacehub_api_token=key1)

# Use the LLM
text = "How does the Large Language Model work?"
output = llm(text)
print(output)

How does the Large Language Model work?

The Large Language Model (LLM) is a type of artificial intelligence (AI) model that is trained on a large amount of text data to generate human-like text. The model is trained on a large corpus of text data, which can include books, articles, and other types of written material. The model is then used to generate new text based on the patterns and relationships it has learned from the training data.

The LLM is a type of natural language processing (NLP) model


In [7]:
modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [8]:
loader = DirectoryLoader("./data", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)


93

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)

461

In [11]:
# import torch
# torch.cuda.empty_cache()
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
torch.cuda.empty_cache()
gc.collect()

2247

In [12]:
from langchain.vectorstores import Chroma
persist_directory = './db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()

In [13]:
import torch
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
dev

'cuda:0'

In [14]:
retriever = vectordb.as_retriever(search_kwargs={'k': 7})

In [15]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k =  5

In [16]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                                       weights=[0.3, 0.7])

In [12]:
ensemble_retriever.get_relevant_documents("scholars admitted in 2019, 17")

[Document(page_content='4. Choose the option ID  \n\uf0b7 For scholars admitted in 2019 , the default option ID is 13  and for the \nscholars admitted in 2020 , the default option ID is 17 . \n\uf0b7 For the scholars admitted in 2018 or before , please select the \nappropriate option ID from t he list. (To know the available \noptions ,  Option Details - Click Here  ) \n5. Select appropriate Course Type  for each course.  \n \nCourse Type   Description  \nResearch Methodology  Research Methodology course', metadata={'page': 1, 'source': 'data/0.pdf'}),
 Document(page_content='scholar shall be the first author with the Guide as the corresponding author. If there is \nan approved Co -Guide, he/she can be a co -author in the publications  \n8.    There shall not be more than two  Guides for a research scholar.  \n  \nResearch Regulations applicable to the scholars who were admitted from 2015 to \n2018  \nPh.D. Regulations 2015 and 2017 shall continue and the following amendment to \nbe in

In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=ensemble_retriever,
                                  return_source_documents=True)
     

In [18]:
def process_llm_response(query):
    llm_response = qa_chain(query)
    return llm_response['result']
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [32]:
query = "What do I do for semester abroad?"
from pprint import pprint
pprint(process_llm_response(query))
# qa_chain(query)

('Use the following pieces of context to answer the question at the end. If '
 "you don't know the answer, just say that you don't know, don't try to make "
 'up an answer.\n'
 '\n'
 'CAPSTONE\tPROJECTS/SAP\t(Semester\tAbroad\tProgram)\t\tQ34.\tWhat\tare\tthe\t'
 'various\tcategories\tto\tdo\tthe\tcapstone\tproject?\t'
 'ANS:\t\t\t\t\t\t\t\t\t\t\t1.\t\t\tIn-House\t(within\tcampus\tunder\tthe\t'
 'faculty\tguidance)\t\t\t\t\t\t\t\t\t\t\t2.\t\t\tCDC\tproject\t(Through\t'
 'CDC)\t\t\t\t\t\t\t\t\t\t\t3.\t\t\tNon-CDC\tProject\t(Capstone\tprojects\tin\t'
 'industry\tbut\tnot\tthrough\tCDC)\t\t\t\t\t\t\t\t\t\t\t4.\t\t\tSAP\tproject\t'
 '(Semester\tAbroad\tprogramme\tthru\tIR\tofdice)\t\tQ35.\tWhich\tare\tthe\t'
 'institutes\tI\tcan\tapply\tfor\tSAP?\tANS:\tPartner\tuniversities\tand\t\t'
 'Top\t500\tQoS\tRanking\tUniversities\t/\n'
 '\n'
 'GRADE\tIMPROVEMENT\t\tQ12:\t\tCan\tI\tdo\tGrade\tImprovement\tin\tsummer\t'
 'Semester\tand\tWeekend\tsemester?\tANS:\tNo,\tGrade\timprovement\tis\t'
 'all

In [ ]:
def main():
    st.title("VIT QA System")
    user_input = st.text_area("Enter Query")

    if st.button("Get response"):
        result = process_llm_response(user_input)
        st.write("Response:")
        # make text bigger
        st.write(f"**{result}**")
main()